In [79]:
def resolvent(C1, C2, common_var):
    if (common_var in C1 and -common_var in C2) or (-common_var in C1 and common_var in C2):
        R = C1.union(C2)
        R.remove(common_var)
        R.remove(-common_var)
        return R
    
    return None

In [80]:
def learning_procedure(AH, conflict_clause, clauses):
    """
    :param AH: Assignment history
    :param conflict_clause:
    :param clauses:
    :return: The learned clause
    """
    k = len(AH)
    X = conflict_clause
    for i in range(k, 0, -1):
        C_num = AH[i-1][0]
        Q = AH[i-1][1]
        if C_num > 0 and resolvent(set(X), set(clauses[C_num]), Q) is not None:
            X = resolvent(set(X), set(clauses[C_num]), Q)

    return X

In [81]:
def is_clause_true(clause, vals):
    result = 0
    for i in clause:
        if i > 0:
            result = result or vals[i]
        else:
            result = result or not(vals[-i])

    return result

def is_clause_unassigned(clause, vals):
    for Pi in clause:
        if vals[abs(Pi)] == -1:
            return True
    return False

In [82]:
# Function that returns a conflict clause if it exists
def get_conflict_clause(vals, clauses):
    for _, clause in clauses.items():
        if not is_clause_unassigned(clause, vals) and not is_clause_true(clause, vals):
            return clause
    return None

In [83]:
def is_unit_clause(clause, vals):
    """
    Checks if a clause is a unit clause.
    Returns (True, unassigned_literal) if it is a unit clause, (False, None) otherwise.
    """
    count_unassigned = 0
    for l in clause:
        if vals[abs(l)] == -1:
            count_unassigned += 1
            unassigned_literal = l
            if count_unassigned > 1: # This means that there is more than one unassigned variable
                return (False, None)
        
        elif l > 0 and vals[abs(l)] == 1: # This means that it's a literal equal to 1
            return (False, None)
        
        elif l < 0 and vals[abs(l)] == 0: # Also, this means that it's a literal equal to 1
            return (False, None)
        
    if count_unassigned == 1:
        return (True, unassigned_literal)
    return (False, None)

In [84]:
def unit_prop(vals, clauses: dict, AH):
    """Unit propagation"""
    found = True
    while found:
      found = False
      for i, clause in clauses.items():
          is_unit, unassigned_lit = is_unit_clause(clause, vals)
          if is_unit:
              found = True
              vals[abs(unassigned_lit)] = 1 if unassigned_lit > 0 else 0
              # Adding it to the assignment history
              AH.append((i, abs(unassigned_lit), 1 if unassigned_lit > 0 else 0))
              

In [85]:
def formula_is_satisfied(clauses, vals):
    for _, clause in clauses.items():
        if not is_clause_true(clause, vals):
            return 0
    return 1

##### Assignment history representation
Each element of the assignment history is represented as a 3-uple.
- Its first element is either 0 (decision) or > 0 (number of a clause)
- Its second element is the variable number
- Its third element is the value we assigned to that variable 

In [86]:
def step4(vals, clauses, AH):
    """Returns 0 if UNSAT, 2 if next step is STEP 2, and 5 if next step is STEP 5"""
    conflict_clause = get_conflict_clause(vals, clauses)
    if conflict_clause is None:
        # There is no conflict clause, so go to step 5
        return 5
    
    learned_clause = learning_procedure(AH, conflict_clause, clauses)

    # if the learned clause is empty, return UNSAT (0)
    if (len(learned_clause) == 0):
        return 0
    
    # else, add the learned clause to F
    # TODO: should i check if the learned clause already exists?
    clauses[len(clauses) + 1] = learned_clause

    # Then, do backtracking on the learned clause
    i = len(AH) - 1

    while not is_unit_clause(learned_clause, vals)[0]:
        var_idx = AH[i][1]
        vals[var_idx] = -1
        del AH[i]

        i -= 1

    # Then, go to step 2
    return 2

# Full DPLL algorithm

In [87]:
def dpll(clauses, n_vars):
    # Initialization
    vals = {i: -1 for i in range(1, n_vars + 1)}
    AH = []

    while True:
        # Unit propagation
        unit_prop(vals, clauses, AH)

        if formula_is_satisfied(clauses, vals):
            return vals
        
        step4_res = step4(vals, clauses, AH)
        if step4_res == 0:
            return 'UNSAT'
        elif step4_res == 2:
            # Go to step 2:
            continue
        elif step4_res == 5:
            # Decision strategy: pick the smallest i such that Pi = -1 and set Pi to 1
            for i in range(1, n_vars + 1):
                if vals[i] == -1:
                    vals[i] = 1
                    AH.append((0, i, 1))
                    break
            continue



Testing


In [105]:
C1 = [-1, 3]
C2 = [-1, 4]
C3 = [-3, -4]
C4 = [-2, 5]
C5 = [-2, 6]
C6 = [-5, -6]

clauses = {
  1: C1, 
  2: C2, 
  3: C3, 
  4: C4, 
  5: C5,
  6: C6,
}

dpll(clauses, 6)

{1: 0, 2: 0, 3: 1, 4: 0, 5: 1, 6: 0}

In [104]:
C1 = [-1, 3]
C2 = [-1, 4]
C3 = [-3, -4]
C4 = [-2, 5]
C5 = [-2, 6]
C6 = [-5, -6]
C7 = [1]

clauses = {
  1: C1, 
  2: C2, 
  3: C3, 
  4: C4, 
  5: C5,
  6: C6,
  7: C7
}

dpll(clauses, 6)

'UNSAT'